In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Helper functions and imports

In [ ]:
from copy import deepcopy
from datetime import datetime
import json
from pathlib import Path

from shutil import copy as shutil_copy

In [ ]:
def write_jobs(jobs):
    now = datetime.now().strftime("%y_%m_%d_%H_%M_%S_jobs")
    target_directory = Path("jobs") / now
    print(f"Writing jobs to {target_directory}")
    target_directory.mkdir(exist_ok=False, parents=True)
    for ii, job in enumerate(jobs):
        path = target_directory / f"{ii:03}"
        path.mkdir(exist_ok=False, parents=False)
        with open(path / "config.json", 'w') as f:
            json.dump(job, f, indent=4, sort_keys=True)
        shutil_copy("run.py", path / "run.py")
        shutil_copy("submit.sbatch", path / "submit.sbatch")

# First pass

We'll just mess around with `Secondary_alcohol` for a bit to start.

`Results` keyword args

In [ ]:
# These go in `rf_kwargs` ###################
ccp_alpha_values = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
max_features_values = [0.3, 0.4, 0.5, 0.6, 0.7]
#############################################

d = {
    "module_path": "/sdcc/u/mcarbone/Github/multimodal-molecules",
    # "module_path": "/Users/mc/GitHub/AIMM/multimodal-molecules",
    "results_kwargs": {
        "conditions": "C-XANES,O-XANES",
        "xanes_data_name": "/hpcgpfs01/work/cfn/mcarbone/multimodal_molecules_data/data/221205/221205_xanes.pkl",
        "index_data_name": "/hpcgpfs01/work/cfn/mcarbone/multimodal_molecules_data/data/221205/221205_index.csv",
        # "xanes_data_name": "/Users/mc/GitHub/AIMM/multimodal-molecules/data/221205/221205_xanes.pkl",
        # "index_data_name": "/Users/mc/GitHub/AIMM/multimodal-molecules/data/221205/221205_index.csv",
        "specific_functional_groups": ["Secondary_alcohol"],
        "val_size": 0.1,
        "test_size": 0.1,
        "rf_kwargs": {"max_samples": 0.9},
        "random_state": 1234,
    }
}


`Results.run_experiment` keyword args

In [ ]:
# Output data directory is relative to the location where the script is run,
# which will be unique
d["run_experiments_kwargs"] = {
    "output_data_directory": "results",
    "n_jobs": 12,
    "compute_feature_importance": False,
}

In [ ]:
jobs = []
for ccp_alpha in ccp_alpha_values:
    for max_features in max_features_values:
        d2 = deepcopy(d)
        d2["results_kwargs"]["rf_kwargs"]["ccp_alpha"] = ccp_alpha
        d2["results_kwargs"]["rf_kwargs"]["max_features"] = max_features
        jobs.append(d2)

In [ ]:
write_jobs(jobs)